<a href="https://colab.research.google.com/github/swaroopkasaraneni/AI/blob/main/TextClassification1Case2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The data for this assignment is taken from Kaggle. (https://www.kaggle.com/zynicide/wine-reviews)

1. Read the CSV file“Wine.csv”
2. Using Pre-process File which you created in module 2 case study 1, call the ‘Refine’ functionand get the pre-processed text for each ‘description’ in the csv file. Store it in a column named “Refined-Description”
3. Using ‘CountVectorization’ function from ‘Vectorization’ python file createdin case study 1of this module,vectorise all the rows in ‘Refined-Description’ column which you created in the above step. Store them in a column named “CountVectorizer”
4. Using ‘TF-IDFVectorization’functionfrom ‘Vectorization’ python file createdin case study 1 of this module,vectorise all the rows in ‘Refined-Description’ column and store the results in ‘TF-IDF Vectorizer Column”
5. Save changes to the CSV file.


In [3]:
%%writefile Vectorization.py

from sklearn.feature_extraction.text import TfidfVectorizer
def TFIDFVectorization(sentences):
    """
    Converts sentences into TF-IDF vectors using scikit-learn's TfidfVectorizer.

    Parameters:
    sentences (list of str): List of input sentences.

    Returns:
    tuple: A tuple containing:
        - feature_names (list of str): List of terms used as features.
        - vectors (list of list of float): List of TF-IDF vectors for each sentence.
    """
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences)
    feature_names = vectorizer.get_feature_names_out()
    # Convert the TF-IDF matrix to a dense list of vectors
    vectors = tfidf_matrix.toarray()
    return feature_names, vectors

def PresenceAbsenceVectorization(sentences):
    """
    Converts sentences into presence/absence vectors based on unique words in the corpus.

    Parameters:
    sentences (list of str): List of input sentences.

    Returns:
    tuple: A tuple containing:
        - corpus (list of str): List of unique words from the input sentences.
        - vectors (list of list of int): List of vectors corresponding to each sentence.
    """
    corpus = set()
    for sentence in sentences:
        words = sentence.split()
        corpus.update(words)
    corpus = sorted(corpus)

    vectors = []
    for sentence in sentences:
        words = sentence.split()
        vector = [1 if word in words else 0 for word in corpus]
        vectors.append(vector)
    return corpus, vectors

def CountVectorization(sentences):
    """
    Converts sentences into count vectors based on unique words in the corpus.

    Parameters:
    sentences (list of str): List of input sentences.

    Returns:
    tuple: A tuple containing:
        - corpus (list of str): List of unique words from the input sentences.
        - vectors (list of list of int): List of count vectors corresponding to each sentence.
    """
    from collections import Counter

    corpus = set()
    for sentence in sentences:
        words = sentence.split()
        corpus.update(words)
    corpus = sorted(corpus)

    vectors = []
    for sentence in sentences:
        word_counts = Counter(sentence.split())
        vector = [word_counts[word] for word in corpus]
        vectors.append(vector)

    return corpus, vectors



Writing Vectorization.py


In [8]:
%%writefile PreProcess.py

from collections import Counter
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer

# List of stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
# lemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

def tokenize(input_string):
    """
    Tokenize the input string into words and return a list of tokens.
    """
    tokens = input_string.split()
    return tokens


# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_words(tokens):
    """
    Lemmatize each word in the string and return the lemmatized string.
    """
    lemmatized = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(lemmatized), lemmatized

def remove_stop_words(tokens):
    """
    Remove stop words from the input string and return the modified string.
    Also, count the frequency of stop words in the string.
    """
    stop_word_counts = Counter(word for word in tokens if word.lower() in stop_words)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return filtered_tokens, stop_word_counts

# Refine function
def Refine(input_string):
    """
    Refines the input string by tokenizing, removing stop words, and lemmatizing it.
    """
    print(f"Input String: {input_string}")
    # Step 1: Tokenize
    tokenized = tokenize(input_string)
    print(f"Tokenized: {tokenized}")

    # Step 2: Remove Stop Words
    removed_stopwords, no_stopwords = remove_stop_words(tokenized)
    print(f"After Stop Words Removal: {removed_stopwords}")

    # Step 3: Lemmatize
    refined_string,lemmatized = lemmatize_words(removed_stopwords)
    print(f"Lemmatized: {lemmatized}")

    return refined_string

Overwriting PreProcess.py


In [7]:
import kagglehub
import pandas as pd

import kagglehub
import pandas as pd
import zipfile
import os

# Download the dataset using kagglehub
path = kagglehub.dataset_download("zynicide/wine-reviews")
print(f"Dataset downloaded to: {path}")

# Unzip the downloaded file if it is compressed
if path.endswith(".zip"):
    with zipfile.ZipFile(path, 'r') as zip_ref:
        extract_path = os.path.dirname(path)
        zip_ref.extractall(extract_path)
    print(f"Dataset extracted to: {extract_path}")

    # Look for CSV files in the extracted directory
    files = os.listdir(extract_path)
    csv_file = next((f for f in files if f.endswith(".csv")), None)

    if csv_file:
        dataset_path = os.path.join(extract_path, csv_file)
        print(f"CSV file found: {dataset_path}")
else:
    dataset_path = path+"/winemag-data_first150k.csv"

# Load the CSV file into a Pandas DataFrame
wine_df = pd.read_csv(dataset_path)

# Display the first few rows
print(wine_df.head())






Dataset downloaded to: /root/.cache/kagglehub/datasets/zynicide/wine-reviews/versions/4
   Unnamed: 0 country                                        description  \
0           0      US  This tremendous 100% varietal wine hails from ...   
1           1   Spain  Ripe aromas of fig, blackberry and cassis are ...   
2           2      US  Mac Watson honors the memory of a wine once ma...   
3           3      US  This spent 20 months in 30% new French oak, an...   
4           4  France  This is the top wine from La Bégude, named aft...   

                            designation  points  price        province  \
0                     Martha's Vineyard      96  235.0      California   
1  Carodorum Selección Especial Reserva      96  110.0  Northern Spain   
2         Special Selected Late Harvest      96   90.0      California   
3                               Reserve      96   65.0          Oregon   
4                            La Brûlade      95   66.0        Provence   

          

In [9]:
from PreProcess import Refine

# Apply the Refine function to the 'description' column
wine_df["Refined-Description"] = wine_df["description"].apply(Refine)

# Check the refined descriptions
print(wine_df["Refined-Description"].head())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


Streaming output truncated to the last 5000 lines.
After Stop Words Removal: ['Surprisingly', 'rich', 'layered', 'price.', 'Shines', 'grass,', 'melon,', 'smoke,', 'lime', 'butter', 'aromas.', 'Equally', 'tasty', 'mouth,', 'spicy', 'density', 'lingers', 'long', 'palate.', 'flavorful,', 'ripe,', 'full-bodied', 'lemon-and-lime', 'fruit.', 'Zesty', 'acidity', 'keeps', 'slight', 'sweetness', 'balanced.', 'One', "year's", 'best', 'Sauvignon', 'Blanc', 'values.', 'Best', 'Buy']
Lemmatized: ['Surprisingly', 'rich', 'layered', 'price.', 'Shines', 'grass,', 'melon,', 'smoke,', 'lime', 'butter', 'aromas.', 'Equally', 'tasty', 'mouth,', 'spicy', 'density', 'lingers', 'long', 'palate.', 'flavorful,', 'ripe,', 'full-bodied', 'lemon-and-lime', 'fruit.', 'Zesty', 'acidity', 'keep', 'slight', 'sweetness', 'balanced.', 'One', "year's", 'best', 'Sauvignon', 'Blanc', 'values.', 'Best', 'Buy']
Input String: A crisp sparkler that features a nice blend of tropical fruit with green apples and limes. Some bisc

In [10]:
from Vectorization import CountVectorization

# Vectorize the refined descriptions
count_vectors = wine_df["Refined-Description"].apply(lambda x: CountVectorization([x])[0])

# Store the vectors in a new column
wine_df["CountVectorizer"] = count_vectors

# Check the count vectors
print(wine_df["CountVectorizer"].head())


0    [100%, 2022–2030., Balanced, Enjoy, Juicy, Oak...
1    [2023., Drink, Ripe, aroma, baking, balanced.,...
2    [122, Dark, Mac, Watson, balanced, botrytised,...
3    [20, 2032., 30%, Abetina, Aromatic,, Aurora,, ...
4    [1200, 18, 2020., Bégude,, Château, Drink, Gis...
Name: CountVectorizer, dtype: object


In [ ]:
from Vectorization import TFIDFVectorization

# Vectorize the refined descriptions
tfidf_features, tfidf_vectors = TFIDFVectorization(wine_df["Refined-Description"].tolist())

# Store the vectors in a new column
wine_df["TF-IDF Vectorizer"] = list(tfidf_vectors)

# Check the TF-IDF vectors
print(wine_df["TF-IDF Vectorizer"].head())


In [ ]:
# Save the modified DataFrame to a new CSV file
wine_df.to_csv("Wine_Refined.csv", index=False)

print("File saved as 'Wine_Refined.csv'.")
